In [7]:
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [8]:
class KMeansSegmentation:
    def __init__(self, pcd, n_clusters=2, height_weight=1.0, slope_weight=1.0, normal_weight=1.0):
        self.pcd = pcd
        self.n_clusters = n_clusters
        self.labels = np.array([])
        self.height_weight = height_weight
        self.slope_weight = slope_weight
        self.normal_weight = normal_weight
        self.pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=100))

        # Adjusting the orientations of the normals to be consistent with each other, based on local tangent planes.
        self.pcd.orient_normals_consistent_tangent_plane(k=4)
        self.pcd.normals = o3d.utility.Vector3dVector(-np.asarray(self.pcd.normals))

    def calculate_slope(self, normals):
        # Assuming that the normals are already normalized
        # The slope can be calculated as the arccosine of the dot product
        # of the normal with the Z-axis (0, 0, 1)
        z_axis = np.array([0, 0, 1])
        slopes = np.arccos(np.clip(np.dot(normals, z_axis), -1.0, 1.0))
        return slopes

    def calculate_curvature(self):
        # Create a KDTree for the point cloud
        pcd_tree = o3d.geometry.KDTreeFlann(self.pcd)

        # Placeholder for curvature values
        curvature = np.zeros(len(np.asarray(self.pcd.points)))

        for i, point in enumerate(self.pcd.points):
            # Search for nearest neighbors of the given point
            [k, idx, _] = pcd_tree.search_radius_vector_3d(point, 0.2)  # Radius 
            
            # Calculate the covariance matrix of the neighborhood
            if k < 3:
                continue
            
            neighbors = np.asarray(self.pcd.points)[idx, :]
            mean = np.mean(neighbors, axis=0)
            covariance_matrix = np.cov((neighbors - mean).T)

            # Eigen decomposition
            eigen_values, _ = np.linalg.eigh(covariance_matrix)
            eigen_values.sort()

            # The curvature can be approximated as the ratio of the smallest
            # to the sum of eigenvalues (Gaussian curvature approximation)
            curvature[i] = eigen_values[0] / np.sum(eigen_values)
        
        return curvature

    def segment(self):
        pcd_points = np.asarray(self.pcd.points)
        normals = np.asarray(self.pcd.normals)

        # Normalize the features
        #normals = (normals - np.mean(normals, axis=0)) / np.std(normals, axis=0)
        min_z = np.min(pcd_points[:, 2])
        relative_height = pcd_points[:, 2] - min_z
        relative_height = (relative_height - np.mean(relative_height)) / np.std(relative_height)
        slopes = self.calculate_slope(normals)
        slopes = (slopes - np.mean(slopes)) / np.std(slopes)

        # Apply feature weights
        relative_height *= self.height_weight
        slopes *= self.slope_weight
        normals *= self.normal_weight

        # Combine features with weights applied
        features = np.hstack((pcd_points[:, :2],
                              relative_height[:, np.newaxis],
                              slopes[:, np.newaxis],
                              normals))

        # Apply K-Means
        kmeans = KMeans(n_clusters=self.n_clusters)
        kmeans.fit(features)

        # Assign clusters
        self.labels = kmeans.labels_
        
        return self.labels

        # Normalize the features all at once
        # features = (features - np.mean(features, axis=0)) / np.std(features, axis=0)

        # Apply K-Means
        kmeans = KMeans(n_clusters=self.n_clusters)
        kmeans.fit(features)

        # Assign clusters
        self.labels = kmeans.labels_
        
        return self.labels


    def visualize_segmentation(self):
        max_label = self.labels.max()
        colors = plt.get_cmap('viridis')(self.labels / (max_label if max_label > 0 else 1))
        colors = colors[:, :3]  # remove the alpha channel
        self.pcd.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([self.pcd],
                                          #point_show_normal = True
                                         )


In [9]:

# Load your point cloud
pcd = o3d.io.read_point_cloud("pbr28.pcd")


# Create an instance of the KMeansSegmentation class
kmeans_segmentation = KMeansSegmentation(pcd, n_clusters=2,
                                         height_weight=2.5, slope_weight=2.0, normal_weight=3 #weights for the features
                                        )

# Perform K-Means segmentation
segment_labels = kmeans_segmentation.segment()

# kmeans_segmentation.post_process_segments(
#     #cluster_size_threshold_ratio=0.15
# )


In [ ]:
# Visualize the segmented point cloud
kmeans_segmentation.visualize_segmentation()